In [1]:
using Muon
using RData
using Revise
using ISCHIA
using DataFrames
using Combinatorics

In [2]:
spatial_object = readh5ad("../data/06-pdac_nac-clusters-rmv_unk.h5ad")
# adata = readh5ad("../data/05-pdac_nac-clusters.h5ad")
lr_network = load("../data/lr_network.rds")

# # Remove spots where neoadjuvant_chemo is unknown
# mask = .!(adata.obs.neoadjuvant_chemo .== "Unknown")
# spatial_object = @view adata[mask, :]
# # spatial_object = adata[mask, :]

spatial_object

AnnData object 216180 ✕ 17764

In [3]:
display(unique(spatial_object.obs.neoadjuvant_chemo))

2-element Vector{String}:
 "No"
 "Yes"

In [12]:
gene_names = collect(spatial_object.var_names)
spatial_object.var.name = gene_names

# Create LR_Pairs column
lr_network[!, :LR_Pairs] = string.(lr_network.from, "_", lr_network.to);
lr_network = lr_network[:, [:from, :to, :LR_Pairs]];

# Filter lr_network to only include genes in adata
from_filter = in.(lr_network[!, :from], Ref(gene_names))
to_filter = in.(lr_network[:, :to], Ref(gene_names))
all_LR_network = lr_network[from_filter .& to_filter, :];

In [13]:
# Extract unique genes and common genes
all_LR_genes = unique(vcat(all_LR_network[:, :from], all_LR_network[:, :to]))
all_LR_genes_comm = intersect(all_LR_genes, collect(gene_names));

# Create LR.pairs and LR.pairs.AllCombos
LR_pairs = all_LR_network[:, :LR_Pairs]
all_combos = [join(combo, "_") for combo in combinations(all_LR_genes_comm, 2)];

In [14]:
# spatial_object = adata
LR_list = all_LR_genes_comm
LR_pairs = LR_pairs
exp_th = 1
corr_th = 0.2;

cc_column = "CC_k10"
cc_list = ["CC10"]
# Condition = unique(spatial_object.obs[!, "orig.ident"])
condition_list = ["Yes", "No"]
condition_column = "neoadjuvant_chemo"

"neoadjuvant_chemo"

In [15]:
for cc in cc_list
    println("Running for $cc")
    for condition in condition_list
        println("Running for $condition")
        lr_result = find_enriched_LR_pairs(
            spatial_object,
            [cc],
            [condition],
            LR_list,
            LR_pairs,
            exp_th,
            corr_th,
            cc_column=cc_column,
            condition_column=condition_column
        )

        CSV.write("outputs/pdac_nac/$(cc)_lr_enrichment_$(condition).csv", lr_result["enriched_LRs"])
        CSV.write("outputs/pdac_nac/$(cc)_cooccurr_mat_$(condition).csv", lr_result["cooccurrence_table"].results)
    end
end

Running for CC10
Running for Yes
Preparing L-R presence/absence matrix
Calculating L-R pairs correlation
Preparing for cooccurrence


Cooccurrence calculation starts...


Calculate Incidence 100%|████████████████████████████████| Time: 0:02:17m
Calculate Co-occurrences 100%|███████████████████████████| Time: 0:02:15
Main Comp 100%|██████████████████████████████████████████| Time: 0:00:43


Cooccurrence calculation ended

Summary of cooccurrence results:
Of 824970 species pair combinations, 573708 pairs (69.54%) were removed from the analysis because expected co-occurrence was < 1 and
251262 pairs were analyzed

Cooccurrence Summary:

Species => 1285
Non-random (%) => 66.3
Sites => 4494
Negative => 2665
Random => 84781
Positive => 163816
Unclassifiable => 0
